In [ ]:
import cv2
import numpy as np
import glob
import os

class cameraProcessor:
    def __init__(self, outpath = 'images'):
        # Variables Calibracion
        self._mtx = []
        self._dist = []
        self._image_res = []

        # Path de saldia de imágenes
        self._outpath = outpath

        # Variables ArUco
        self._center_coord = []

    def ImageFileName(img_path):
        images = []
        for ext in ('*.jpg', '*.png', '*.jpeg'):
            images.extend(glob.glob(os.path.join(img_path, ext)))
        return images
 
    def calib(self, calib_img_path, chess_dim = (9,6), chess_square_lenght = 30.0):
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corner_dim = (chess_dim[0]-1, chess_dim[1]-1)    #Las funciones utilizan las esquinas internas, no la cant de cuadrados

        objp = np.zeros((corner_dim[0]*corner_dim[1], 3), np.float32)
        objp[:,:2] = np.mgrid[0:corner_dim[0],0:corner_dim[1]].T.reshape(-1,2)
        objp *= chess_square_lenght     # Grilla con todas las posiciones de las esquinas

        objpoints = [] # puntos 3D en el mundo real
        imgpoints = [] # puntos 2D en la imagen

        # Lista de imagenes
        images = self.ImageFileName(calib_img_path)

        if not images:
            print("No hay imagenes.")
            return None
        if len(images) < 10:
            print("Advertencia. Pocas imágenes. Se recomiendan 10 o más en distintos ángulos")

        for fname in images:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)		# Lo pasa a escala de grises para mejorar
            ret, corners = cv2.findChessboardCornersSB(gray, corner_dim, None)
            
            if ret == True:
                print(f"Encontrado en {fname}")
                objpoints.append(objp)
                corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
                imgpoints.append(corners2)

        if len(objpoints) == 0:
            print("No se detectaron esquinas en ninguna imagen.")
            return None

        # Resolucion utilizada para calibrar
        img = cv2.imread(images[0])                  # Nota = usa solo la primer imagen !!! 
        self._image_res = (img.shape[1], img.shape[0])    # calib_res = (WIDHT, HEIGHT) 

        # Calibración
        ret, self._mtx, self._dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        
        return self._mtx, self._dist, rvecs, tvecs, self._image_res
    
    def saveCalibMatrix(self, file_name='calib_matrix'):
        out_path = os.path.join(self._outpath, file_name)
        try:
            np.savez(out_path, mtx=self._mtx, dist=self._dist, _image_res=self._image_res )
            print(f"Matrices guardadas en {file_name}.npz")
        except Exception as e:
            print(f"Error al guardar: {e}")

    def loadCalibMatrix(self, file_name='calib_matrix'):
        out_path = os.path.join(self._outpath, file_name)
        try:
            calib = np.load(f"{out_path}.npz")
            self._mtx, self._dist, self._image_res = calib["mtx"], calib["dist"], calib['_image_res']
        except Exception as e:
            print('Error al cargar el archivo')
        return self._mtx, self._dist, self._image_res

    def undistort(self, img):
        try:
            h,  w = img.shape[:2]
            newcameramtx, roi = cv2.getOptimalNewCameraMatrix(self._mtx, self._dist, (w,h), 1, (w,h))

            # Elimina distorcion
            undistorted = cv2.undistort(img, self._mtx, self._dist, None, newcameramtx)
    
            # Recorte de la imagen
            x, y, w, h = roi
            undistorted = undistorted[y:y+h, x:x+w]
            return undistorted, newcameramtx, roi
        except Exception as e:
            print(f'Error al corregir distorcion {e}')
            return None, None, None
        

    def warp(self, img, corner_ids, plane_size, pixels_per_mm = 2):
        '''
        Realiza Homografía con 4 esquinas ArUco.
        calib_value_file: Nombre de archivo (sin extensión) de las matrices de distorsión.
        images_path: path a carpeta de imágenes que van a realizar el path.
        corner_ids: IDs de ArUco a identificar como esquinas.
        plane_size: Tamaño del plano de 4 esquinas.
        pixels_per_mm: Resolución pixeles --> mm.
        '''
        # Cálculo de puntos de esquinas (ejemplo: ([0 0], [0 100], [24 0], [24 100]) mm)
        image_size = (plane_size[0]*pixels_per_mm,
                    plane_size[1]*pixels_per_mm) 

        real_points = np.array([
                        [0, 0],                             # arriba izq
                        [image_size[0]-1, 0],               # arriba der
                        [image_size[0]-1, image_size[1]-1], # abajo der
                        [0, image_size[1]-1]                # abajo izq
                    ], dtype=np.float32)

        # Detector de ArUco
        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
        parameters = cv2.aruco.DetectorParameters()
        detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

        if self._mtx is None or self._dist is None:
            print("No hay matriz intrinseca y/o coeficientes de distorsion")
            return


        if [img.shape[1], img.shape[0]] != self._image_res:
            print("WARNING: Resolucion de imagen diferente a la de calibracion\n")

       
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = detector.detectMarkers(gray_img)

        corner_coords = self.get_corners(corner_ids, corners, ids)
        if corner_coords is None or len(corner_coords) != 4:
            print('no hay coordenadas')
            return None

        # Corrección de la imagen: 
        H, _ = cv2.findHomography(corner_coords, real_points)
        aligned_img = cv2.warpPerspective(img, H, image_size)

        return aligned_img

    def get_corners(self, corner_ids, corners, ids):
        '''
        Devuelve un vector de 4 posiciones con los centros de cada esquina. 
        El resultado está ordenado [TOP LEFT, TOP RIGHT, BOTTOM RIGHT, BOTTOM LEFT]
        corner_ids: Id's a utilizar como esquina.
        corners: Vectos de esquinas de todos los ArUco obtenidos.
        ids: Id de cada ArUco enviado en corners.
        '''
        detected_centers = []
        corner_coords = []
        if ids is not None:
            # Busqueda de corners:
            ids = ids.flatten()

            for i, marker_id in enumerate(ids):
                # Obtengo centros solo de los corners ID:
                if marker_id in corner_ids:
                    c = corners[i][0]
                    center = c.mean(axis=0)
                    detected_centers.append(center)

            if len(detected_centers) == 4:
                detected_centers = np.array(detected_centers, dtype=np.float32)

                s = detected_centers.sum(axis=1)
                diff = np.diff(detected_centers, axis=1)

                top_left = detected_centers[np.argmin(s)]
                bottom_right = detected_centers[np.argmax(s)]
                top_right = detected_centers[np.argmin(diff)]
                bottom_left = detected_centers[np.argmax(diff)]

                corner_coords = np.array([top_left, top_right, bottom_right, bottom_left], dtype=np.float32)

        return corner_coords

    def colorFilter(self, img, color_range):
        '''
        Detecta regiones de un color específico en todas las imágenes dentro de 'images_path/warped'
        y guarda los resultados en 'images_path/color'.

        Parameters:
            images_path (str): Carpeta base donde están las imágenes originales.
            color_range (tuple): (lower_HSV, upper_HSV) con los límites del color a detectar.
            show (bool): Muestra los resultados visualmente (por defecto True).
        '''

        if not img:
            print("No hay imágenes.")
            return

        lower, upper = color_range


        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower, upper)

        # Limpieza de ruido
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # resultado a color
        masked_img_color = cv2.bitwise_and(img, img, mask=mask)

        return mask, masked_img_color

    def loadImage(self, img_path):
        img = cv2.imread(img_path)
        return img
    
    def saveImage(self, img, file_name='img.jpeg', img_rel_path=''):
        out_path = os.path.join(self._outpath, img_rel_path)
        out_path = os.path.join(out_path, file_name)
        cv2.imwrite(out_path, img)

    def processImages(self, img_folder_path, color_filter, corner_ids, plane_size, pixels_per_mm = 2,  save = False, out_path='processed'):
        images = self.ImageFileName(img_folder_path)
        img_res = []
        img_color_res = []

        for fname in images:
            img = self.loadImage(fname)
            img = self.undistort(img)
            img = self.warp(img, corner_ids, plane_size, pixels_per_mm)
            img, img_color = self.colorFilter(img, color_filter)
            img_res.append(img)
            img_color_res.append(img_color)

            if save:
                out = os.path.join(self._outpath, out_path)
                out_bw = os.path.join(out, os.path.basename(fname))
                out_color = os.path.join(out, os.path.basename(fname).replace(".", "_color."))
                cv2.imwrite(out_bw, img)
                cv2.imwrite(out_color, img_color)
        
        return img_res, img_color_res
